In [1]:
import requests
import json
import pandas as pd
import sqlite3
import numpy as np

In [2]:
# Define the API endpoint URL
# We will use a public test API to get a list of posts
api_url = 'https://data.seattle.gov/resource/33kz-ixgy.json' #police calls

# read in data from API to a dataframe
df_seattle = pd.read_json(api_url)

In [3]:
df_seattle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 46 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   cad_event_number                                    1000 non-null   int64         
 1   cad_event_clearance_description                     1000 non-null   object        
 2   call_type                                           1000 non-null   object        
 3   priority                                            1000 non-null   int64         
 4   initial_call_type                                   1000 non-null   object        
 5   final_call_type                                     1000 non-null   object        
 6   cad_event_original_time_queued                      1000 non-null   object        
 7   cad_event_arrived_time                              937 non-null    datetime64[ns]
 8   dispatch_

In [4]:
df_township = pd.DataFrame()
df_responseunit = pd.DataFrame()
df_category = pd.DataFrame()

df_township['name'] = df_seattle['dispatch_neighborhood'].unique()
df_responseunit['station_name'] = df_seattle['dispatch_precinct'].unique()
df_category['name'] = df_seattle['initial_call_type'].unique()

df_township = df_township.reset_index()
df_responseunit = df_responseunit.reset_index()
df_category = df_category.reset_index()

df_township = df_township.rename(columns={'index': 'id'})
df_responseunit = df_responseunit.rename(columns={'index': 'id'})
df_category = df_category.rename(columns={'index': 'id'})
df_responsetype = pd.DataFrame({'id':[0],'name':['POLICE']})

In [5]:
df_emergencycall = df_seattle[['cad_event_original_time_queued','dispatch_address',
     'dispatch_latitude','dispatch_longitude']]
df_emergencycall = df_emergencycall.reset_index()

# generate category_id col
cat_id = []
for v in df_seattle['initial_call_type']:
    cat_id.append(df_category[df_category['name'] == v].id.iloc[0])
df_emergencycall['category_id'] = cat_id

# generate response_unit_id col
ru_id = []
for v in df_seattle['dispatch_precinct']:
    ru_id.append(df_responseunit[df_responseunit['station_name'] == v].id.iloc[0])
df_emergencycall['response_unit_id'] = ru_id
df_emergencycall['zip_code'] = np.NaN

# generate township_id col
ts_id = []
for v in df_seattle['dispatch_neighborhood']:
    ts_id.append(df_township[df_township['name'] == v].id.iloc[0])
df_emergencycall['township_id'] = ts_id

# view the DF
df_emergencycall

,index,cad_event_original_time_queued,dispatch_address,dispatch_latitude,dispatch_longitude,category_id,response_unit_id,zip_code,township_id
0,0,2017-01-25T11:14:52.000,9XX BLOCK OF N 77 ST,47.68476561,-122.34585468,0,0,NaN,0
1,1,2017-01-25T14:29:56.000,12XX BLOCK OF S KING ST,47.59833443,-122.31531757,1,1,NaN,1
2,2,2024-08-14T16:46:04.000,12XX BLOCK OF S WELLER ST,47.59749563,-122.31499334,2,1,NaN,1
3,3,2017-01-25T21:36:03.000,42XX BLOCK OF SW ALASKA ST,47.56108917,-122.38611809,3,2,NaN,2
4,4,2024-08-16T15:35:12.000,11XX BLOCK OF HIAWATHA PL S,47.59229464,-122.30748656,4,3,NaN,3
...,...,...,...,...,...,...,...,...,...
995,995,2021-09-06T01:08:05.000,52XX BLOCK OF 45 AV SW,47.55529368,-122.38972980,21,2,NaN,2
996,996,2021-09-07T12:43:12.000,100XX BLOCK OF AURORA AV N,47.70179210,-122.34465537,153,0,NaN,12
997,997,2021-09-11T06:13:08.000,44XX BLOCK OF 41 AV S,47.56350107,-122.28171391,4,4,NaN,22
998,998,2021-09-13T04:56:12.000,5XX BLOCK OF NE NORTHGATE WY,47.70858578,-122.32191838,38,0,NaN,12


In [6]:
# write to sqlite db
conn = sqlite3.connect('seattledb.sqlite3')
df_emergencycall.to_sql('dashboard_emergencycall', conn, if_exists='replace', index=False)
df_township.to_sql('dashboard_township', conn, if_exists='replace', index=False)
df_responseunit.to_sql('dashboard_responseunit', conn, if_exists='replace', index=False)
df_category.to_sql('dashboard_category', conn, if_exists='replace', index=False)
df_responsetype.to_sql('dashboard_responsetype', conn, if_exists='replace', index=False)

conn.commit()
conn.close()